In [1]:
from transformers import pipeline , AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F
from utils import tokenize_hfmodel_inputs

/home/vamsi/Desktop/projects/AI projects/Testing_diff_models/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# chatbot = pipeline("text-generation", 
#                    model="google/gemma-2b",
#                    torch_dtype=torch.float16,
#                    device_map="auto",
#                    )
# response = chatbot("sample_text", max_new_tokens=50)
# print(response[0]["generated_text"])

In [8]:
low_compute=['mistralai/Mistral-7B-v0.3','google/gemma-2b','microsoft/phi-2','meta-llama/Llama-2-7b-hf','teknium/OpenHermes-2.5-Mistral-7B','meta-llama/Llama-3.1-8B','microsoft/Phi-3-mini-128k-instruct']
instruct_counter_part=['mistralai/Mistral-7B-Instruct-v0.3']

In [ ]:
med_compute=['mistralai/Mixtral-8x7B-v0.1','meta-llama/Llama-2-13b-hf','CohereForAI/c4ai-command-r-plus-08-2024','']
med_instruct_counter_part=['mistralai/Mixtral-8x7B-Instruct-v0.1']

In [10]:
large_compute=['tiiuae/falcon-180B',]
large_instruct_counter_part=['tiiuae/falcon-180B-chat','meta-llama/Llama-3.3-70B-Instruct','meta-llama/Llama-2-70b-chat-hf','mistralai/Mistral-Large-Instruct-2411']

In [11]:
low_coding_models=['meta-llama/CodeLlama-7b-Instruct-hf','bigcode/starcoder2-7b','microsoft/Phi-3-mini-128k-instruct']
med_coding_models=['codellama/CodeLlama-13b-hf','bigcode/starcoder2-15b']
high_coding_models=['mistralai/Codestral-22B-v0.1','codellama/CodeLlama-34b-Instruct-hf','deepseek-ai/deepseek-coder-33b-instruct']

Text completions

In [12]:
model_name="google/gemma-2b"

In [13]:
model=AutoModelForCausalLM.from_pretrained(model_name)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.53s/it]


In [ ]:
sample_text="How to mak a sandwich"
# tokeniser=Autotokeniser.from_pretrained("google/gemma-2b")
# tokens=torch.tensor(tokeniser.encode(sample_text))
# tokens=torch.tensor(tokens).view(1,tokens.shape[0])

In [6]:
tokens,tokeniser=tokenize_hfmodel_inputs(sample_text,model_name,'cpu')

/home/vamsi/Desktop/projects/AI projects/Testing_diff_models/utils.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens=torch.tensor(tokens).view(1,tokens.shape[0])


In [ ]:
# messages = [
#             {"role": "user", "content": sample_text}
#         ]
# prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# input_ids = tokeniser(prompt, return_tensors="pt")
# tkns=input_ids.input_ids

In [ ]:
x=tokens
for _ in range(50):
    ops=model(input_ids=x)
    logits=ops.logits
    probs=F.softmax(logits[:,-1,:],dim=-1)

    next_token=torch.multinomial(probs.view(probs.shape[1]),1)

    x=torch.cat([x,next_token.view(1,1)],dim=-1)

In [17]:
tokeniser.decode(next_token.tolist()[0])

' chung'

In [18]:
op=''

In [19]:
for i in x.tolist():
    op=op+tokeniser.decode(i)

In [20]:
print(op)

<bos><bos><start_of_turn>user
How to mak a sandwich<end_of_turn>
<start_of_turn>model
Followers Clause
 contribuvendo á my fb
 Andrew August 31, 2019
 Proyect Storey of a since 2019
 Trendsetouts
 Townhouse whos konyhok
 konkeyấp chung


In [ ]:
# ops=model.generate(tokens,max_length=50)

In [2]:
# device = "cuda" if torch.cuda.is_available() else "cpu"

Chat

In [2]:
model_name="google/gemma-2b-it"

In [3]:
model=AutoModelForCausalLM.from_pretrained(model_name)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.11s/it]


In [4]:
sample_text="write a c program to print first 5 numbers of fibonacci series."

In [5]:
tokens,tokeniser=tokenize_hfmodel_inputs(sample_text,model_name,'cpu',is_chat=True)

/home/vamsi/Desktop/projects/AI projects/Testing_diff_models/utils.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens=torch.tensor(tokens).view(1,tokens.shape[0])


In [6]:
messages = [
            {"role": "user", "content": sample_text}
        ]
prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

input_ids = tokeniser(prompt, return_tensors="pt")

In [7]:
input_ids.input_ids.shape

torch.Size([1, 24])

In [8]:
output_ids = model.generate(input_ids.input_ids,max_length=200,do_sample=True,temperature=0.7,top_p=0.9)

In [10]:
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)
print(response)

user
write a c program to print first 5 numbers of fibonacci series.
model
```c
#include <stdio.h>

int main()
{
    int a = 0, b = 1, c;

    for (int i = 1; i <= 5; i++)
    {
        printf("%d ", a);
        a = b;
        b = c;
    }

    return 0;
}
```

**Output:**

```
0 1 1 2 3
```

**Explanation:**

* The program uses three variables, `a`, `b`, and `c`, to represent the three consecutive numbers in the Fibonacci sequence.
* The `for` loop iterates from `i = 1` to `i <= 5`.
* Inside the loop, it swaps the values of `a` and `b`


In [12]:
# User asks first question
messages.append({"role": "assistant", "content": response})  # Save model response
messages.append({"role": "user", "content": "But you did not write c=a+b anywhere"})  # New user query

# Reformat input
prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokeniser(prompt, return_tensors="pt")

# Generate again
output_ids = model.generate(input_ids.input_ids, max_length=1200, do_sample=True, top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)

print(response)


user
write a c program to print first 5 numbers of fibonacci series.
model
user
write a c program to print first 5 numbers of fibonacci series.
model
```c
#include <stdio.h>

int main()
{
    int a = 0, b = 1, c;

    for (int i = 1; i <= 5; i++)
    {
        printf("%d ", a);
        a = b;
        b = c;
    }

    return 0;
}
```

**Output:**

```
0 1 1 2 3
```

**Explanation:**

* The program uses three variables, `a`, `b`, and `c`, to represent the three consecutive numbers in the Fibonacci sequence.
* The `for` loop iterates from `i = 1` to `i <= 5`.
* Inside the loop, it swaps the values of `a` and `b`
user
But this is wrong answer
model
user
write a c program to print first 5 numbers of fibonacci series.
model
user
write a c program to print first 5 numbers of fibonacci series.
model
```c
#include <stdio.h>

int main()
{
    int a = 0, b = 1, c;

    for (int i = 1; i <= 5; i++)
    {
        printf("%d ", a);
        a = b;
        b = c;
    }

    return 0;
}
```

**Output: